In [2]:
import pandas as pd
from connections import AWS
from sklearn.model_selection import train_test_split

$\textbf{Epidemiology: Clinical Train/Test Splits}$

Separate cohort of matches into train and validation sets for model development. 

In [3]:
# setup AWS connection
aws = AWS()
aws.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


In [5]:
# load model cohort
model_cohort = aws.load_s3_object('epidemiology/ml/datasets/full/cohort_matches_final.csv')

In [ ]:
# get train/test split (test size: 25%)
    # NOTE: this preserves matching pairs within the split
train_ids, test_ids = train_test_split(
    model_cohort[['mlbamid_injured', 'mlbamid_noninjured']].values, 
    test_size=0.25, 
    random_state=22
)

# restructure to dataframe
train_df = pd.DataFrame(train_ids.reshape(-1, 1)).rename(columns={0: 'mlbamid'}).astype(int)
test_df = pd.DataFrame(test_ids.reshape(-1, 1)).rename(columns={0: 'mlbamid'}).astype(int)

In [34]:
# upload to S3
path_stem = 'epidemiology/ml/datasets/full'
aws.upload_to_s3(train_df, f'{path_stem}/cohort_train_ids.csv')
aws.upload_to_s3(test_df, f'{path_stem}/cohort_test_ids.csv')

[AWS]: Uploaded object to s3://pitch-ml/epidemiology/ml/datasets/full/cohort_train_ids.csv
[AWS]: Uploaded object to s3://pitch-ml/epidemiology/ml/datasets/full/cohort_test_ids.csv


In [35]:
# close AWS connection
aws.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
